### BOAZ 엔지니어 세션 
2022.10 우아라

### Titanic 데이터 EDA 수행.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark
!pip install -q findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 56 kB/s 
     |████████████████████████████████| 199 kB 85.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=686647e1f7731b6a91e0902d18388745eaf04446515ef3f3f4b0c1663818883b
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("learn-sql").getOrCreate()

import findspark
findspark.init()

spark.version

'3.3.0'

In [5]:
# spark.read.csv() 메소드를 이용하여 csv 파일을 로드하고 DataFrame으로 변환. 
# pandas_df = pd.read_csv('/content/drive/MyDrive/SparkML/titanic/train.csv', header='infer')
titanic_sdf = spark.read.csv('/content/drive/MyDrive/SparkML/titanic/train.csv', header=True, inferSchema=True)

# pandas DataFrame을 spark DataFrame으로 부터 생성. 
titanic_pdf = titanic_sdf.select('*').toPandas()

# display(titanic_sdf.limit(10))
titanic_sdf.limit(10).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

* Passengerid: 탑승자 데이터 일련번호
* survived: 생존 여부, 0 = 사망, 1 = 생존
* Pclass: 티켓의 선실 등급, 1 = 일등석, 2 = 이등석, 3 = 삼등석
* sex: 탑승자 성별
* name: 탑승자 이름
* Age: 탑승자 나이
* sibsp: 같이 탑승한 형제자매 또는 배우자 인원수
* parch: 같이 탑승한 부모님 또는 어린이 인원수
* ticket: 티켓 번호
* fare: 요금
* cabin: 선실 번호
* embarked: 중간 정착 항구 C = Cherbourg, Q = Queenstown, S = Southampton

In [6]:
titanic_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int32  
 1   Survived     891 non-null    int32  
 2   Pclass       891 non-null    int32  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int32  
 7   Parch        891 non-null    int32  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int32(5), object(5)
memory usage: 66.3+ KB


In [7]:
# titanic_sdf 의 schema 확인
titanic_sdf.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [8]:
# 컬럼별로 Null 인 경우만 count하는 select 로직.
import pyspark.sql.functions as F

titanic_sdf.select([F.count(F.when(F.col(c).isNull()|F.isnan(c), c)).alias(c) for c in titanic_sdf.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [9]:
titanic_sdf.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [10]:
print("#### Survived value count #### ")
print(titanic_sdf.groupBy('Survived').count().show())
print("#### Pclass value count #####")
print(titanic_sdf.groupBy('Pclass').count().show())

#### Survived value count #### 
+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+

None
#### Pclass value count #####
+------+-----+
|Pclass|count|
+------+-----+
|     1|  216|
|     3|  491|
|     2|  184|
+------+-----+

None


In [11]:
groupby_cols = ['Survived', 'Pclass', 'Sex', 'Ticket', 'Cabin', 'Embarked']

for groupby_col in groupby_cols:
    print('##### ' + groupby_col + ' value counts ######')
    titanic_sdf.groupBy(groupby_col).count().show()

##### Survived value counts ######
+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+

##### Pclass value counts ######
+------+-----+
|Pclass|count|
+------+-----+
|     1|  216|
|     3|  491|
|     2|  184|
+------+-----+

##### Sex value counts ######
+------+-----+
|   Sex|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+

##### Ticket value counts ######
+----------------+-----+
|          Ticket|count|
+----------------+-----+
|          367230|    2|
|       P/PP 3381|    2|
|          244270|    1|
|          363291|    3|
|SOTON/OQ 3101317|    1|
|           31418|    1|
|           26360|    2|
|            2700|    1|
|           14313|    1|
|          345763|    1|
|       A/5 21172|    1|
|          364500|    1|
|           19877|    2|
|          350029|    1|
|          113800|    1|
|      A/4. 39886|    1|
|       C.A. 2673|    2|
|          113807|    1|
|        PC 17604|    2|
|       C.A. 2315| 

### 내장 그래프 기능을 활용한 EDA
* Spark DataFrame은 matplotlib 등의 파이썬 시각화 라이브러리와 바로 연동할 수 없음. 
* 내장된 그래프 기능을 통해 SQL 수행 결과를 다양한 그래프로 시각화 할 수 있음. 
* Spark DataFrame을 pandas API on spark(Spark 3.2 이상)을 이용할 경우 파이썬 시각화 라이브러리와 연동 할 수 있음.

In [13]:
# Pandas API on Spark 적용
import pyspark.pandas as ps

# spark DataFrame에서 Spark pandas DataFrame으로 변경. 
# titanic_sdf.select('Age').hist()는 동작하지 않음. 

psdf = titanic_sdf.to_pandas_on_spark() # pandas DataFrame으로 부터 변경할 경우는 psdf = ps.from_pandas(titanic_pdf) 
print(type(psdf))

psdf['Age'].hist()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:3315: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  FutureWarning,


<class 'pyspark.pandas.frame.DataFrame'>


In [14]:
# SQL 사용을 위해서 Spark DataFrame을 View형태로 변환
titanic_sdf.createOrReplaceTempView('titanic_view')

In [17]:
# survived groupBy 수 계산
spark.sql('select survived, count(*) as cnt from titanic_view group by survived order by 2 desc').show()

+--------+---+
|survived|cnt|
+--------+---+
|       0|549|
|       1|342|
+--------+---+



In [18]:
# cabin groupBy 수 계산
spark.sql('select cabin, count(*) as cnt from titanic_view group by cabin order by 2 desc').show()

+-----------+---+
|      cabin|cnt|
+-----------+---+
|       null|687|
|    B96 B98|  4|
|         G6|  4|
|C23 C25 C27|  4|
|       E101|  3|
|    C22 C26|  3|
|          D|  3|
|        F33|  3|
|         F2|  3|
|        E33|  2|
|       C123|  2|
|        C83|  2|
|       C126|  2|
|       E121|  2|
|    B58 B60|  2|
|        D35|  2|
|      F G73|  2|
|        B18|  2|
|        B20|  2|
|        E44|  2|
+-----------+---+
only showing top 20 rows



SQL로 Histogram을 구하고 이를 시각화

In [25]:
# age_bin_range 컬럼 만들어서 나잇대별 인원수 확인

spark.sql('''
select age_bin, age_bin||'-'||(age_bin+10) as age_bin_range, count(*)  cnt
from (
  select floor(age/10.0)*10 as age_bin from titanic_view
) group by age_bin order by 3 desc;

''').show()

+-------+-------------+---+
|age_bin|age_bin_range|cnt|
+-------+-------------+---+
|     20|        20-30|220|
|   null|         null|177|
|     30|        30-40|167|
|     10|        10-20|102|
|     40|        40-50| 89|
|      0|         0-10| 62|
|     50|        50-60| 48|
|     60|        60-70| 19|
|     70|        70-80|  6|
|     80|        80-90|  1|
+-------+-------------+---+



In [28]:
titanic_sdf.groupBy('Survived', 'Sex').count().show()

+--------+------+-----+
|Survived|   Sex|count|
+--------+------+-----+
|       0|female|   81|
|       1|  male|  109|
|       1|female|  233|
|       0|  male|  468|
+--------+------+-----+



In [30]:
spark.sql('''
select survived, sex, count(*) from titanic_view group by survived, sex
''').show()

+--------+------+--------+
|survived|   sex|count(1)|
+--------+------+--------+
|       0|female|      81|
|       1|  male|     109|
|       1|female|     233|
|       0|  male|     468|
+--------+------+--------+



In [32]:
spark.sql('''
select pclass, sex, survived, count(*) cnt from titanic_view group by pclass, sex, survived order by 1, 2, 3
''').show()

+------+------+--------+---+
|pclass|   sex|survived|cnt|
+------+------+--------+---+
|     1|female|       0|  3|
|     1|female|       1| 91|
|     1|  male|       0| 77|
|     1|  male|       1| 45|
|     2|female|       0|  6|
|     2|female|       1| 70|
|     2|  male|       0| 91|
|     2|  male|       1| 17|
|     3|female|       0| 72|
|     3|female|       1| 72|
|     3|  male|       0|300|
|     3|  male|       1| 47|
+------+------+--------+---+



In [34]:
# 조건에 해당되는 내용만 count.

spark.sql('''
select pclass, sex, 
sum(case when survived = 0 then 1 else 0 end) cnt_non_survived,
sum(case when survived = 1 then 1 else 0 end) cnt_survived from titanic_view group by pclass, sex order by 1, 2
''').show()

+------+------+----------------+------------+
|pclass|   sex|cnt_non_survived|cnt_survived|
+------+------+----------------+------------+
|     1|female|               3|          91|
|     1|  male|              77|          45|
|     2|female|               6|          70|
|     2|  male|              91|          17|
|     3|female|              72|          72|
|     3|  male|             300|          47|
+------+------+----------------+------------+



In [35]:
# 일반 python용 UDF를 작성. 반드시 입력 값과 반환 값을 설정
def get_category(age):
    cat = ''
    
    # age 값이 None일 경우는 NA를 Return (Pandas와 다르게 null을 처리해야 함. None인 이유는 python이기 때문)
    if age == None:
        return 'NA'
    
    if age <= 5: cat = 'Baby'
    elif age <= 12: cat = 'Child'
    elif age <= 18: cat = 'Teenager'
    elif age <= 25: cat = 'Student'
    elif age <= 35: cat = 'Young Adult'
    elif age <= 60: cat = 'Adult'
    else : cat = 'Elderly'
    
    return cat

from pyspark.sql.functions import udf,col
from pyspark.sql.types import StringType

# 일반 python용 UDF를 pyspark용 UDF로 변환. udf(lambda 입력변수: 일반 UDF, 해당 일반 UDF의 반환형)
udf_get_category = udf(lambda x:get_category(x), StringType() )
# udf_get_category()에 Age 컬럼값을 입력하여 반환되는 값으로 새로운 컬럼 Age_Category를 생성

titanic_sdf = titanic_sdf.withColumn("Age_Category",udf_get_category(col("Age")))
titanic_sdf.show()

'''
#아래와 같이 when otherwise 를 이용해서도 변환 가능. 
from pyspark.sql.functions import when
                   
titanic_sdf_filled_02 = titanic_sdf.withColumn('Age_category', when(F.col('Age') <= 5, 'Baby')
                                                                      .when(F.col('Age') <= 12, 'Child')
                                                                      .when(F.col('Age') <= 18, 'Teenage')
                                                                      .when(F.col('Age') <= 25, 'Student')
                                                                      .when(F.col('Age') <= 35, 'Young Adult')
                                                                      .when(F.col('Age') <= 60, 'Adult')
                                                                      .when(F.col('Age').isNull(), 'NA')
                                                                      .otherwise('Elderly'))
'''

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Age_Category|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|     Student|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|       Adult|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S| Young Adult|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S| Young Adult|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null

"\n#아래와 같이 when otherwise 를 이용해서도 변환 가능. \nfrom pyspark.sql.functions import when\n                   \ntitanic_sdf_filled_02 = titanic_sdf.withColumn('Age_category', when(F.col('Age') <= 5, 'Baby')\n                                                                      .when(F.col('Age') <= 12, 'Child')\n                                                                      .when(F.col('Age') <= 18, 'Teenage')\n                                                                      .when(F.col('Age') <= 25, 'Student')\n                                                                      .when(F.col('Age') <= 35, 'Young Adult')\n                                                                      .when(F.col('Age') <= 60, 'Adult')\n                                                                      .when(F.col('Age').isNull(), 'NA')\n                                                                      .otherwise('Elderly'))\n"

In [36]:
#-- 새로운 컬럼을 추가시 View에 자동 반영되지 않음. 
spark.sql('''
select * from titanic_view
''').show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [37]:
titanic_sdf.createOrReplaceTempView('titanic_view')

In [39]:
spark.sql('''
select * from titanic_view
''').show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Age_Category|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|     Student|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|       Adult|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S| Young Adult|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S| Young Adult|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null

In [40]:
spark.sql('''
select age_category, survived, sex, count(*) cnt from titanic_view group by  age_category, survived, sex order by cnt desc
''').show()


+------------+--------+------+---+
|age_category|survived|   sex|cnt|
+------------+--------+------+---+
|          NA|       0|  male|108|
|       Adult|       0|  male|101|
| Young Adult|       0|  male|100|
|     Student|       0|  male| 95|
|       Adult|       1|female| 54|
| Young Adult|       1|female| 53|
|     Student|       1|female| 41|
|          NA|       1|female| 36|
|    Teenager|       0|  male| 31|
| Young Adult|       1|  male| 30|
|    Teenager|       1|female| 27|
|       Adult|       1|  male| 24|
|     Elderly|       0|  male| 17|
|          NA|       0|female| 17|
|          NA|       1|  male| 16|
|        Baby|       1|female| 16|
|       Adult|       0|female| 16|
|        Baby|       1|  male| 15|
|     Student|       1|  male| 13|
|     Student|       0|female| 13|
+------------+--------+------+---+
only showing top 20 rows



### 결측치 처리, 컬럼 가공, 인코딩 수행.

In [41]:
import pyspark.sql.functions as F

avg_age = titanic_sdf.select(F.avg(F.col('Age')))
avg_age_row = avg_age.first() # avg_age.head()와 동일. 
avg_age_value = avg_age.first()[0]

print(avg_age, avg_age_row, avg_age_value)
print(avg_age.show())

DataFrame[avg(Age): double] Row(avg(Age)=29.69911764705882) 29.69911764705882
+-----------------+
|         avg(Age)|
+-----------------+
|29.69911764705882|
+-----------------+

None


In [42]:
titanic_sdf = titanic_sdf.fillna({'Age':titanic_sdf.select(F.avg(F.col('Age'))).first()[0],
                   'Cabin': 'N',
                   'Embarked': 'N'
                   })
titanic_sdf.limit(20).show()

+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+------------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Age_Category|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25|    N|       S|     Student|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|  C85|       C|       Adult|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0|    0|STON/O2. 3101282|  7.925|    N|       S| Young Adult|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|    0|          113803|   53.1| C123|       S| Young Adult|
|          5|

In [43]:
# age의 Null값을 평균으로 대체했으므로 Age_Category 컬럼도 재 생성
titanic_sdf = titanic_sdf.withColumn("Age_Category",udf_get_category(col("Age")))

(titanic_sdf.groupBy('age_category', 'survived', 'sex').count()).show()


+------------+--------+------+-----+
|age_category|survived|   sex|count|
+------------+--------+------+-----+
|     Elderly|       1|female|    3|
|     Student|       1|female|   41|
|    Teenager|       0|  male|   31|
|    Teenager|       0|female|    9|
|     Student|       1|  male|   13|
|        Baby|       0|female|    5|
|       Child|       0|female|    8|
|     Student|       0|female|   13|
| Young Adult|       0|female|   30|
|     Elderly|       0|  male|   17|
|    Teenager|       1|female|   27|
|       Child|       1|  male|    6|
|        Baby|       1|  male|   15|
| Young Adult|       1|  male|   46|
|        Baby|       0|  male|    8|
|       Adult|       1|female|   54|
|       Adult|       0|  male|  101|
|     Student|       0|  male|   95|
|       Child|       1|female|    3|
|       Adult|       1|  male|   24|
+------------+--------+------+-----+
only showing top 20 rows



In [44]:
# Cabin 컬럼은 맨 앞자리 문자로 대체. 컬럼 update시에 withColumn() 메소드 적용. 
titanic_sdf = titanic_sdf.withColumn('Cabin', F.substring(F.col('Cabin'), 0, 1))
(titanic_sdf.limit(10)).show()

+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+------------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Age_Category|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25|    N|       S|     Student|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|    C|       C|       Adult|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0|    0|STON/O2. 3101282|  7.925|    N|       S| Young Adult|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|    0|          113803|   53.1|    C|       S| Young Adult|
|          5|

In [45]:
def fill_n_transform_columns(titanic_sdf):
    titanic_sdf = titanic_sdf.fillna({'Age':titanic_sdf.select(F.avg(F.col('Age'))).first()[0],
                                      'Cabin': 'N',
                                      'Embarked':'N' 
                                     })

    titanic_sdf = titanic_sdf.withColumn("Age_Category",udf_get_category(col("Age")))
    titanic_sdf = titanic_sdf.withColumn('Cabin', F.substring(F.col('Cabin'), 0, 1))
    
    return titanic_sdf

titanic_sdf = fill_n_transform_columns(titanic_sdf)
(titanic_sdf.limit(10)).show()

+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+------------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Age_Category|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25|    N|       S|     Student|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|    C|       C|       Adult|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0|    0|STON/O2. 3101282|  7.925|    N|       S| Young Adult|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|    0|          113803|   53.1|    C|       S| Young Adult|
|          5|

In [46]:
# null 값이 있는지 재 확인. 
titanic_sdf.select([F.count(F.when(F.col(c).isNull()|F.isnan(c), c)).alias(c) for c in titanic_sdf.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+------------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|Age_Category|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+------------+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    0|       0|           0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+------------+



In [47]:
# 특정 컬럼들을 삭제하기. VectorAssembler 로 변환된 feature vector만 학습/예측에 참여하므로 여기서 삭제하지 않고 나중에 feature vectorization 때 제외해도 무방. 
def drop_columns(sdf, drop_cols=None):
    # drop_cols는 인자로 list가 입력되나 Spark DataFrame의 drop() 메소드는 인자로 컬럼명 문자열을 입력 받음. 
    sdf = sdf.drop(*drop_cols)
    return sdf

titanic_sdf = drop_columns(titanic_sdf, drop_cols=['PassengerId', 'Name', 'Ticket'])

(titanic_sdf.limit(10)).show()

+--------+------+------+-----------------+-----+-----+-------+-----+--------+------------+
|Survived|Pclass|   Sex|              Age|SibSp|Parch|   Fare|Cabin|Embarked|Age_Category|
+--------+------+------+-----------------+-----+-----+-------+-----+--------+------------+
|       0|     3|  male|             22.0|    1|    0|   7.25|    N|       S|     Student|
|       1|     1|female|             38.0|    1|    0|71.2833|    C|       C|       Adult|
|       1|     3|female|             26.0|    0|    0|  7.925|    N|       S| Young Adult|
|       1|     1|female|             35.0|    1|    0|   53.1|    C|       S| Young Adult|
|       0|     3|  male|             35.0|    0|    0|   8.05|    N|       S| Young Adult|
|       0|     3|  male|29.69911764705882|    0|    0| 8.4583|    N|       Q| Young Adult|
|       0|     1|  male|             54.0|    0|    0|51.8625|    E|       S|       Adult|
|       0|     3|  male|              2.0|    3|    1| 21.075|    N|       S|        Baby|

In [48]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

def label_encode_columns(sdf, input_cols=None):
    
    output_cols = [ 'label_'+col for col in input_cols]
    string_indexer = StringIndexer(inputCols=input_cols, outputCols=output_cols)
    sdf = string_indexer.fit(sdf).transform(sdf)
    
    return sdf

def onehot_encode_columns(sdf, input_cols=None):
    
    label_cols = ['label_'+col for col in input_cols]
            
    string_indexer = StringIndexer(inputCols=input_cols, outputCols=label_cols)
    sdf = string_indexer.fit(sdf).transform(sdf)
    # sdf = label_encode_columns(sdf, input_cols)
    
    output_cols = ['onehot_'+col for col in input_cols]
    onehot_encoder = OneHotEncoder(inputCols=label_cols, outputCols=output_cols)
    sdf = onehot_encoder.fit(sdf).transform(sdf)
    # one hot encoding 하기 위해 생성한 label encoding된 컬럼을 삭제
    sdf = sdf.drop(*label_cols)
    
    return sdf

In [49]:
titanic_sdf_enc_01 = label_encode_columns(titanic_sdf.select('*'), input_cols=['Age_Category', 'Cabin', 'Sex', 'Embarked'])
titanic_sdf_enc_01.show()

+--------+------+------+-----------------+-----+-----+-------+-----+--------+------------+------------------+-----------+---------+--------------+
|Survived|Pclass|   Sex|              Age|SibSp|Parch|   Fare|Cabin|Embarked|Age_Category|label_Age_Category|label_Cabin|label_Sex|label_Embarked|
+--------+------+------+-----------------+-----+-----+-------+-----+--------+------------+------------------+-----------+---------+--------------+
|       0|     3|  male|             22.0|    1|    0|   7.25|    N|       S|     Student|               2.0|        0.0|      0.0|           0.0|
|       1|     1|female|             38.0|    1|    0|71.2833|    C|       C|       Adult|               1.0|        1.0|      1.0|           1.0|
|       1|     3|female|             26.0|    0|    0|  7.925|    N|       S| Young Adult|               0.0|        0.0|      1.0|           0.0|
|       1|     1|female|             35.0|    1|    0|   53.1|    C|       S| Young Adult|               0.0|        1

In [50]:
titanic_sdf_enc_01 = onehot_encode_columns(titanic_sdf.select('*'), input_cols=['Age_Category', 'Cabin', 'Sex', 'Embarked'])
(titanic_sdf_enc_01).show()

+--------+------+------+-----------------+-----+-----+-------+-----+--------+------------+-------------------+-------------+-------------+---------------+
|Survived|Pclass|   Sex|              Age|SibSp|Parch|   Fare|Cabin|Embarked|Age_Category|onehot_Age_Category| onehot_Cabin|   onehot_Sex|onehot_Embarked|
+--------+------+------+-----------------+-----+-----+-------+-----+--------+------------+-------------------+-------------+-------------+---------------+
|       0|     3|  male|             22.0|    1|    0|   7.25|    N|       S|     Student|      (6,[2],[1.0])|(8,[0],[1.0])|(1,[0],[1.0])|  (3,[0],[1.0])|
|       1|     1|female|             38.0|    1|    0|71.2833|    C|       C|       Adult|      (6,[1],[1.0])|(8,[1],[1.0])|    (1,[],[])|  (3,[1],[1.0])|
|       1|     3|female|             26.0|    0|    0|  7.925|    N|       S| Young Adult|      (6,[0],[1.0])|(8,[0],[1.0])|    (1,[],[])|  (3,[0],[1.0])|
|       1|     1|female|             35.0|    1|    0|   53.1|    C|  

In [51]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# Pipeline을 이용한 Encoding 함수. 
def encode_columns(sdf, input_cols=None, encode_gubun='label'):
    # label encoding과 one hot encoding 변환 컬럼명 지정. 
    label_cols = ['label_'+col for col in input_cols]
    onehot_cols = ['onehot_'+col for col in input_cols]
    
    #pipeline의 stages로 지정된 StringIndexer와 OneHotEncoder 객체 생성. 
    label_encoder_stage = StringIndexer(inputCols=input_cols, outputCols=label_cols)
    onehot_encoder_stage = OneHotEncoder(inputCols=label_cols, outputCols=onehot_cols)
    
    # encode_gubun이 label이면 StringIndexer stage만 등록, onehot이면 StringIndexer, OneHotEncoder 모두 등록. 
    stages = []
    if encode_gubun == 'label':
        stages = [label_encoder_stage]
    else:
        stages = [label_encoder_stage, onehot_encoder_stage]
        
    pipeline = Pipeline(stages=stages)
    sdf = pipeline.fit(sdf).transform(sdf)
    
    return sdf
  
titanic_sdf_encode_02 = encode_columns(titanic_sdf.select('*'), input_cols=['Age_Category', 'Cabin', 'Sex', 'Embarked'], encode_gubun='onehot')
(titanic_sdf_encode_02.limit(10)).show()

+--------+------+------+-----------------+-----+-----+-------+-----+--------+------------+------------------+-----------+---------+--------------+-------------------+-------------+-------------+---------------+
|Survived|Pclass|   Sex|              Age|SibSp|Parch|   Fare|Cabin|Embarked|Age_Category|label_Age_Category|label_Cabin|label_Sex|label_Embarked|onehot_Age_Category| onehot_Cabin|   onehot_Sex|onehot_Embarked|
+--------+------+------+-----------------+-----+-----+-------+-----+--------+------------+------------------+-----------+---------+--------------+-------------------+-------------+-------------+---------------+
|       0|     3|  male|             22.0|    1|    0|   7.25|    N|       S|     Student|               2.0|        0.0|      0.0|           0.0|      (6,[2],[1.0])|(8,[0],[1.0])|(1,[0],[1.0])|  (3,[0],[1.0])|
|       1|     1|female|             38.0|    1|    0|71.2833|    C|       C|       Adult|               1.0|        1.0|      1.0|           1.0|      (6,[

### 학습, 예측, 평가 수행. 
* 데이터 재로딩 및 사전 데이터 재 가공 수행. 
* feature vectorization 적용 후 학습과 테스트 데이터 세트로 분할하여 학습, 예측, 성능 평가 수행.

In [52]:
# titanic 데이터 세트를 다시 spark DataFrame으로 재로딩하고 데이터 사전 가공, Encoding 작업 진행. 
titanic_sdf = spark.read.csv('/content/drive/MyDrive/SparkML/titanic/train.csv', header=True, inferSchema=True)

titanic_sdf = fill_n_transform_columns(titanic_sdf)
titanic_sdf = encode_columns(titanic_sdf, input_cols=['Age_Category', 'Cabin', 'Sex', 'Embarked'], encode_gubun='label')

# schema 확인 및 Null 값 확인. 
titanic_sdf.printSchema()
titanic_sdf.select([F.count(F.when(F.col(c).isNull()|F.isnan(c), c)).alias(c) for c in titanic_sdf.columns]).show()
(titanic_sdf.limit(10)).show()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = false)
 |-- Embarked: string (nullable = false)
 |-- Age_Category: string (nullable = true)
 |-- label_Age_Category: double (nullable = false)
 |-- label_Cabin: double (nullable = false)
 |-- label_Sex: double (nullable = false)
 |-- label_Embarked: double (nullable = false)

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+------------+------------------+-----------+---------+--------------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|Age_Category|label_Age_Category|label_Cabin|label_Sex|label_Embarked|
+---

In [53]:
# feature vectorization 적용할 column명 추출. 문자열 컬럼 제외, label 컬럼인 Survived는 제외, 불필요한 PassengerId 컬럼 제외
drop_columns = [column_name for column_name, column_type in titanic_sdf.dtypes if column_type == 'string']
drop_columns += ['Survived', 'PassengerId']
print('삭제될 columns:', drop_columns)

# titanic_sdf Dataframe 컬럼에서 삭제될 컬럼을 제외.  
vector_columns = [column for column in titanic_sdf.columns if column not in drop_columns]
print('vector화될 columns:', vector_columns)

삭제될 columns: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age_Category', 'Survived', 'PassengerId']
vector화될 columns: ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'label_Age_Category', 'label_Cabin', 'label_Sex', 'label_Embarked']


In [56]:
# feature vector화 적용 후 train과 test 데이터 세트로 분리. 
from pyspark.ml.feature import VectorAssembler

vector_assembler = VectorAssembler(inputCols=vector_columns, outputCol='features')
titanic_sdf = vector_assembler.transform(titanic_sdf)

# Pipeline을 적용하지 않을 것이면 전체 데이터를 feature vector화 한 후에 train과 test 데이터 세트로 분리하는게 조금 더 편리. 
train_sdf , test_sdf = titanic_sdf.randomSplit([0.8, 0.2], seed=11)
(train_sdf.limit(10)).show()

# 만약, Pipeline은 안에서 vectorization을 알아서 수행하기 때문에 미리 쪼갤 필요는 없음. 하지만 다른 패키지를 사용한다면, 전처리 작업을 모두 한 후 분리하는게 더 편리.

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------------+------------------+-----------+---------+--------------+--------------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Age_Category|label_Age_Category|label_Cabin|label_Sex|label_Embarked|            features|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------------+------------------+-----------+---------+--------------+--------------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|    N|       S|     Student|               2.0|        0.0|      0.0|           0.0|[3.0,22.0,1.0,0.0...|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|    C|       C|       Adult|               1.0|        1.0|      1.0|           1

In [58]:
# 학습 데이터로 학습 후 테스트 데이터로 예측 
from pyspark.ml.classification import DecisionTreeClassifier

dt_estimator = DecisionTreeClassifier(featuresCol='features', labelCol='Survived')
dt_model = dt_estimator.fit(train_sdf)
# test_sdf는 이미 feature vector화 되어 있으므로 별도의 feature vector화 작업 불필요. 
predictions = dt_model.transform(test_sdf)

(predictions).show()

+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------+-------+-----+--------+------------+------------------+-----------+---------+--------------+--------------------+-------------+--------------------+----------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|    Ticket|   Fare|Cabin|Embarked|Age_Category|label_Age_Category|label_Cabin|label_Sex|label_Embarked|            features|rawPrediction|         probability|prediction|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------+-------+-----+--------+------------+------------------+-----------+---------+--------------+--------------------+-------------+--------------------+----------+
|          6|       0|     3|    Moran, Mr. James|  male|29.69911764705882|    0|    0|    330877| 8.4583|    N|       Q| Young Adult|               0.0|        0.0|      0.0|           2.0|(9,[0,1,4,8],[3.0...| [302.0,37.0]|[

In [59]:
# 테스트 데이터의 예측 성능 정확도 평가
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

accuracy_evaluator = MulticlassClassificationEvaluator(labelCol='Survived', predictionCol='prediction', metricName='accuracy')
print('정확도:', accuracy_evaluator.evaluate(predictions))


정확도: 0.8297872340425532


In [60]:
dt_estimator.__class__.__name__

'DecisionTreeClassifier'

In [61]:
# estimator 별로 학습/예측/평가를 수행하기 위한 함수 생성. 
def train_test_eval(estimator, train_sdf, test_sdf, evaluator):
    estimator_model = estimator.fit(train_sdf)
    predictions = estimator_model.transform(test_sdf)
    accuracy = evaluator.evaluate(predictions)
    print(estimator.__class__.__name__, ' 정확도:', accuracy)

# 이미 titanic_sdf에 features 컬럼이 있으면 이를 drop
titanic_sdf = titanic_sdf.drop('features')

vector_assembler = VectorAssembler(inputCols=vector_columns, outputCol='features')
titanic_sdf = vector_assembler.transform(titanic_sdf)
train_sdf , test_sdf = titanic_sdf.randomSplit([0.8, 0.2], seed=11)

dt_estimator = DecisionTreeClassifier(featuresCol='features', labelCol='Survived')
accuracy_evaluator = MulticlassClassificationEvaluator(labelCol='Survived', predictionCol='prediction', metricName='accuracy')

train_test_eval(dt_estimator, train_sdf, test_sdf, accuracy_evaluator)
    

DecisionTreeClassifier  정확도: 0.8297872340425532


In [62]:
# 여러 classifier를 학습 후 성능 테스트 
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier

# 학습/예측/평가를 수행한 Estimator 객체들을 생성. 결정 트리, 랜덤 포레스트, 로지스틱 회귀로 테스트 
estimators = [DecisionTreeClassifier(featuresCol='features', labelCol='Survived'),
              RandomForestClassifier(featuresCol='features', labelCol='Survived'),
              LogisticRegression(featuresCol='features', labelCol='Survived')
             ]

for estimator in estimators:
    train_test_eval(estimator, train_sdf, test_sdf, accuracy_evaluator)

DecisionTreeClassifier  정확도: 0.8297872340425532
RandomForestClassifier  정확도: 0.8457446808510638
LogisticRegression  정확도: 0.7872340425531915


### 교차 검증과 하이퍼 파라미터 튜닝 수행.

In [63]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

dt_estimator = DecisionTreeClassifier(featuresCol='features', labelCol='Survived')

#CrossValidator에서 하이퍼파라미터 튜닝을 위한 그리드 서치(Grid Search)용 ParamGrid 생성.
# Spark ML DecisionTreeClassifier의 maxDepth는 max_depth, minInstancesPerNode는 min_samples_split(노드 분할 시 최소 sample 건수)
dt_param_grid = ParamGridBuilder().addGrid(dt_estimator.maxDepth, [2, 3, 5, 7, 10])\
                               .addGrid(dt_estimator.minInstancesPerNode, [2, 3, 4,  5])\
                               .build()
accuracy_evaluator = MulticlassClassificationEvaluator(labelCol='Survived', predictionCol='prediction', metricName='accuracy')

# scikit-learn의 gridSearchCV 와 비슷
cv = CrossValidator(estimator=dt_estimator, estimatorParamMaps=dt_param_grid, evaluator=accuracy_evaluator, numFolds=3)
cv_model = cv.fit(train_sdf)

In [65]:
import pandas as pd

# 교차 검증 결과를 pandas DataFrame으로 반환하는 함수 생성. 
def get_cv_result_pdf(cv_model):
    params = [{p.name: v for p, v in m.items()} for m in cv_model.getEstimatorParamMaps()]

    cv_result_pdf= pd.DataFrame({'params': params, 'evaluation_result':cv_model.avgMetrics })
    
    return cv_result_pdf

result_pdf = get_cv_result_pdf(cv_model)
result_pdf

,params,evaluation_result
0,"{'maxDepth': 2, 'minInstancesPerNode': 2}",0.775507
1,"{'maxDepth': 2, 'minInstancesPerNode': 3}",0.775507
2,"{'maxDepth': 2, 'minInstancesPerNode': 4}",0.775507
3,"{'maxDepth': 2, 'minInstancesPerNode': 5}",0.775507
4,"{'maxDepth': 3, 'minInstancesPerNode': 2}",0.794100
5,"{'maxDepth': 3, 'minInstancesPerNode': 3}",0.795608
6,"{'maxDepth': 3, 'minInstancesPerNode': 4}",0.795608
7,"{'maxDepth': 3, 'minInstancesPerNode': 5}",0.795608
8,"{'maxDepth': 5, 'minInstancesPerNode': 2}",0.800497
9,"{'maxDepth': 5, 'minInstancesPerNode': 3}",0.800401


In [66]:
#test_sdf는 이미 feature vector화 되어 있음. 
predictions = cv_model.transform(test_sdf)

print("\n##### cv_model로 테스트 데이터 예측 결과 ######")
accuracy_evaluator = MulticlassClassificationEvaluator(labelCol='Survived', predictionCol='prediction', metricName='accuracy')
print('테스트 데이터 세트 정확도:', accuracy_evaluator.evaluate(predictions))

(predictions).show()


##### cv_model로 테스트 데이터 예측 결과 ######
테스트 데이터 세트 정확도: 0.8085106382978723
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------+-------+-----+--------+------------+------------------+-----------+---------+--------------+--------------------+-------------+--------------------+----------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|    Ticket|   Fare|Cabin|Embarked|Age_Category|label_Age_Category|label_Cabin|label_Sex|label_Embarked|            features|rawPrediction|         probability|prediction|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------+-------+-----+--------+------------+------------------+-----------+---------+--------------+--------------------+-------------+--------------------+----------+
|          6|       0|     3|    Moran, Mr. James|  male|29.69911764705882|    0|    0|    330877| 8.4583|    N|       Q| Young Adult|               0.0|

In [67]:
def cv_train_test_eval(train_sdf, test_sdf, estimator, param_grid, evaluator, num_folds):
    cv = CrossValidator(estimator=estimator, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=num_folds)
    cv_model = cv.fit(train_sdf)
    cv_result_pdf = get_cv_result_pdf(cv_model)
    
    predictions = cv_model.transform(test_sdf)
    test_accuracy = evaluator.evaluate(predictions)
    
    return cv_result_pdf, test_accuracy
    
cv_result_pdf, test_accuracy = cv_train_test_eval(train_sdf, test_sdf, 
                                                  dt_estimator, dt_param_grid, accuracy_evaluator, num_folds=5)
print('테스트 데이터 세트 정확도:', test_accuracy)
(cv_result_pdf)

    

테스트 데이터 세트 정확도: 0.7925531914893617


,params,evaluation_result
0,"{'maxDepth': 2, 'minInstancesPerNode': 2}",0.765892
1,"{'maxDepth': 2, 'minInstancesPerNode': 3}",0.765892
2,"{'maxDepth': 2, 'minInstancesPerNode': 4}",0.765892
3,"{'maxDepth': 2, 'minInstancesPerNode': 5}",0.765892
4,"{'maxDepth': 3, 'minInstancesPerNode': 2}",0.818079
5,"{'maxDepth': 3, 'minInstancesPerNode': 3}",0.819654
6,"{'maxDepth': 3, 'minInstancesPerNode': 4}",0.819654
7,"{'maxDepth': 3, 'minInstancesPerNode': 5}",0.815544
8,"{'maxDepth': 5, 'minInstancesPerNode': 2}",0.810455
9,"{'maxDepth': 5, 'minInstancesPerNode': 3}",0.809085
